In [ ]:
# install virgin environment
!odsc conda install --uri <oci://my-bucket@my-namespace/path_to_tar_file>

# modify environment
!pip install ...

# publish in project-specific bucket final env with project-specific name
!odsc conda init -b <proj-specific-bucket-name> -n <your-tenancy-namespace> -a resource_principal
!odsc conda publish -s <slug> --force

Here we have to define the infrastructure operations for the first deploy and maybe the update of the infrastructure if a re-deploy is needed

* creation of log group and custom log to be used in data science pipeline
* creation of data science pipeline --> here we should in future define a wrapper to let data scientist define if to use 1 or more steps
* creation of project - task - application - publishing task to application - scheduler - scheduled task

in future maybe also add the monitoring of infrastructure

In [ ]:
import oci
from oci.logging.models import CreateLogDetails, UpdateLogDetails, CreateLogGroupDetails
import requests

# Initialize the signer with Resource Principal
resource_principal_signer = oci.auth.signers.get_resource_principals_signer()

### LOGGING STEP

In [ ]:
# CREATE LOG GROUP
# Create a logging client with the signer
log_client = oci.logging.LoggingManagementClient({}, signer=resource_principal_signer)

log_group = log_client.create_log_group(
    CreateLogGroupDetails(
        compartment_id="ocid1.compartment.oc1..aaaaaaaagds646vklqyxsj2t2lmaexhpraeak44qmyyg2rymurl45svpwcla",
        display_name="test-iaas-log-group",
        defined_tags={"Sisal":{"environment":"dev","factory":"Digital","country":"Italy"}}
    )
)


# RETRIEVE OCID FOR THE CREATED LOG GROUP
list_log_groups_response = log_client.list_log_groups( 
    compartment_id="ocid1.compartment.oc1..aaaaaaaagds646vklqyxsj2t2lmaexhpraeak44qmyyg2rymurl45svpwcla",
    sort_by="displayName",
    sort_order="DESC")

for log_group in list_log_groups_response.data:    
    if log_group.display_name == "test-iaas-log-group":
        print(f"log group ocid: {log_group.id}")
        log_group_id = log_group.id


# CREATE CUSTOM LOG
log = log_client.create_log(
    log_group_id=log_group_id,
    create_log_details=CreateLogDetails(
        display_name="test-iaas-custom-log",
        log_type="CUSTOM",
        retention_duration=30,
        defined_tags={"Sisal":{"environment":"dev","factory":"Digital","country":"Italy"}}
    )
)


# RETRIEVE CUSTOM LOG OCID
list_log_groups_response = log_client.list_logs( 
    log_group_id=log_group_id,
    sort_by="displayName",
    sort_order="DESC")

for log_group in list_log_groups_response.data:    
    if log_group.display_name == "test-iaas-custom-log":
        print(f"log ocid: {log_group.id}")
        log_group_id = log_group.id

### PIPELINE DEFINITION

### DATA INTEGRATION 

In [ ]:
# CREATE PROJECT
# Initialize the OCI Data Integration client with the signer
data_integration = oci.data_integration.DataIntegrationClient({}, signer=resource_principal_signer)

# Step 1: Create a Data Integration project
project = data_integration.create_project(
    workspace_id="ocid1.test.oc1..<unique_ID>EXAMPLE-workspaceId-Value",
    create_project_details=oci.data_integration.models.CreateProjectDetails(
        name="test-iaas-project"
    )
)

# CREATE REST TASK
# Step 2: Create a REST task inside the project
pipeline_id = "YOUR_PIPELINE_ID"  # Replace with the actual pipeline ID
defined_tags = {"key1": "value1", "key2": "value2"}

rest_task = data_integration.create_task(
    create_task_details=oci.data_integration.models.CreateTaskDetails(
        name="test-rest-task",
        model_type="REST",
        details=oci.data_integration.models.CreateDataIntegrationTaskFromRestTask(
            endpoint="https://datascience.eu-frankfurt-1.oci.oraclecloud.com/20190101/pipelineRuns",
            method=oci.data_integration.models.CreateRestTaskDetails.MethodType.POST,
            request_headers={"Content-Type": "application/json"},
            request_body=oci.data_integration.models.CreateRestTaskFromDataLoaderTaskRestModel(
                projectId=project.id,
                compartmentId=compartment_id,
                pipelineId=pipeline_id,
                displayName=display_name,
                definedTags=defined_tags
            )
        )
    ),
    project.id
)

# VALIDATE REST TASK
# Create an Oracle Cloud Identity Client
identity_client = oci.identity.IdentityClinent({}, signer=resource_principal_signer)

# Get the compartment OCID where your task is located
compartment_id = "YOUR_COMPARTMENT_OCID"

# Use the Oracle Identity client to get the task details
task = identity_client.get_task(compartment_id, rest_task.id)
print("Task Status:", task.data.lifecycle_state)

# Now, you can make a REST API request to the task's endpoint to further validate it
task_endpoint = task.data.endpoint
response = requests.get(task_endpoint)
if response.status_code == 200:
    print("Task is valid. Response:", response.text)
else:
    print("Task is not valid. Response:", response.text)



# CREATE APPLICATION
# Send the request to service, some parameters are not required, see API doc for more info
create_application_response = data_integration.create_application(
    workspace_id="ocid1.test.oc1..<unique_ID>EXAMPLE-workspaceId-Value",
    create_application_details=oci.data_integration.models.CreateApplicationDetails(
        name="EXAMPLE-name-Value",
        identifier="EXAMPLE-identifier-Value",
        description="EXAMPLE-description-Value",
        defined_tags={'EXAMPLE_KEY_eROvo': {'EXAMPLE_KEY_kbfvy': 'EXAMPLE--Value'}}
        )
    )
# Get the data from response
print(create_application_response.data)

# PUBLISH REST TASK IN APPLICATION


# CREATE SCHEDULER
# Send the request to service, some parameters are not required, see API doc for more info
create_schedule_response = data_integration.create_schedule(
    workspace_id="ocid1.test.oc1..<unique_ID>EXAMPLE-workspaceId-Value",
    application_key=create_application_response.id,
    create_schedule_details=oci.data_integration.models.CreateScheduleDetails(
        name="EXAMPLE-name-Value",
        identifier="EXAMPLE-identifier-Value",
        description="EXAMPLE-description-Value",
        frequency_details=oci.data_integration.models.CustomFrequencyDetails(
            model_type="CUSTOM",
            frequency="CUSTOM",
            customExpression="cron expression"
            ),
        timezone="EXAMPLE-timezone-Value"
        )
    )

# Get the data from response
print(create_schedule_response.data)

# CREATE SCHEDULED TASK
# Send the request to service, some parameters are not required, see API doc for more info
create_task_schedule_response = data_integration.create_task_schedule(
    workspace_id="ocid1.test.oc1..<unique_ID>EXAMPLE-workspaceId-Value",
    application_key=create_application_response.id,
    create_task_schedule_details=oci.data_integration.models.CreateTaskScheduleDetails(
        name="EXAMPLE-name-Value",
        identifier="EXAMPLE-identifier-Value",
        description="EXAMPLE-description-Value",
        schedule_ref=oci.data_integration.models.Schedule(
            key="EXAMPLE-key-Value",
            model_version="EXAMPLE-modelVersion-Value",
            model_type="EXAMPLE-modelType-Value",
            parent_ref=oci.data_integration.models.ParentReference(
                    parent="EXAMPLE-parent-Value",
                    root_doc_id="ocid1.test.oc1..<unique_ID>EXAMPLE-rootDocId-Value"),
            name="EXAMPLE-name-Value",
            description="EXAMPLE-description-Value",
            object_version=402,
            object_status=605,
            identifier="EXAMPLE-identifier-Value",
            frequency_details=oci.data_integration.models.WeeklyFrequencyDetails(
                        model_type="WEEKLY",
                        frequency="WEEKLY",
                        time=oci.data_integration.models.Time(
                            hour=6,
                            minute=51,
                            second=22),
                days=["MONDAY"]),
            timezone="EXAMPLE-timezone-Value",
            is_daylight_adjustment_enabled=False,
            metadata=oci.data_integration.models.ObjectMetadata(
                created_by="EXAMPLE-createdBy-Value",
                created_by_name="EXAMPLE-createdByName-Value",
                updated_by="EXAMPLE-updatedBy-Value",
                updated_by_name="EXAMPLE-updatedByName-Value",
                time_created=datetime.strptime(
                    "2009-11-24T23:09:21.007Z",
                    "%Y-%m-%dT%H:%M:%S.%fZ"),
                time_updated=datetime.strptime(
                    "2041-04-27T03:30:15.167Z",
                    "%Y-%m-%dT%H:%M:%S.%fZ"),
                aggregator_key="EXAMPLE-aggregatorKey-Value",
                aggregator=oci.data_integration.models.AggregatorSummary(
                    type="EXAMPLE-type-Value",
                    key="EXAMPLE-key-Value",
                    name="EXAMPLE-name-Value",
                    identifier="EXAMPLE-identifier-Value",
                    description="EXAMPLE-description-Value"),
                identifier_path="EXAMPLE-identifierPath-Value",
                info_fields={
                    'EXAMPLE_KEY_fwqHO': 'EXAMPLE_VALUE_CxirQ8IspoIiS04UegPD'},
                registry_version=914,
                labels=["EXAMPLE--Value"],
                is_favorite=True,
                count_statistics=oci.data_integration.models.CountStatistic(
                    object_type_count_list=[
                        oci.data_integration.models.CountStatisticSummary(
                            object_type="USER_DEFINED_FUNCTION",
                            object_count=259)]))),
        number_of_retries=402,
        retry_delay=2511.9067,
        retry_delay_unit="MINUTES",
        start_time_millis=457,
        auth_mode="RESOURCE_PRINCIPAL"
        )
    )

# Get the data from response
print(create_task_schedule_response.data)